In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, roc_auc_score, recall_score, 
                             precision_score, f1_score, cohen_kappa_score,
                             matthews_corrcoef, confusion_matrix, classification_report,
                             roc_curve, auc)
from sklearn.dummy import DummyClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import zscore, kruskal, mannwhitneyu
from scipy import stats
from tqdm import tqdm
import random
import warnings
import xgboost as xgb
import umap
import logging
from datetime import datetime
import os

warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'microbiome_classification_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


In [43]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def clr_transform(X, pseudocount=0.5):
    """
    Center log-ratio transformation for compositional data
    """
    X_pseudo = X + pseudocount
    geometric_mean = np.exp(np.log(X_pseudo).mean(axis=1, keepdims=True))
    return np.log(X_pseudo / geometric_mean)

# ============================================================================
# NEW: Custom Feature Selector Class for Pipeline
# ============================================================================
class CompositionalFeatureSelector(BaseEstimator, TransformerMixin):
    """
    Feature selector for compositional data with multiple selection methods
    """
    def __init__(self, method='mutual_info', max_features=500, alpha=0.05, 
                 correlation_threshold=0.95, use_fdr=True):
        self.method = method
        self.max_features = max_features
        self.alpha = alpha
        self.correlation_threshold = correlation_threshold
        self.use_fdr = use_fdr
        self.selected_features_ = None
        self.feature_scores_ = None
        
    def fit(self, X, y):
        """Select features based on chosen method"""
        X_df = pd.DataFrame(X)
        y_series = pd.Series(y)
        
        if self.method == 'mutual_info':
            # Use mutual information for compositional data
            from sklearn.feature_selection import mutual_info_classif
            mi_scores = mutual_info_classif(X_df, y_series, random_state=RANDOM_SEED)
            self.feature_scores_ = mi_scores
            
            # Select top features
            if len(mi_scores) > self.max_features:
                threshold = np.sort(mi_scores)[-self.max_features]
                self.selected_features_ = np.where(mi_scores >= threshold)[0]
            else:
                self.selected_features_ = np.arange(len(mi_scores))
                
        elif self.method == 'mannwhitney':
            # Original Mann-Whitney method with improvements
            mw_pvalues = []
            tested_features = []
            
            for col_idx in range(X_df.shape[1]):
                try:
                    healthy_vals = X_df.loc[y_series == 0, col_idx].dropna()
                    disease_vals = X_df.loc[y_series == 1, col_idx].dropna()
                    
                    if len(healthy_vals) < 5 or len(disease_vals) < 5:
                        continue
                    
                    # Check for variance
                    if healthy_vals.std() <= 1e-6 or disease_vals.std() <= 1e-6:
                        continue
                    
                    _, p_val = mannwhitneyu(healthy_vals, disease_vals, alternative="two-sided")
                    mw_pvalues.append(p_val)
                    tested_features.append(col_idx)
                except Exception as e:
                    continue
            
            if len(tested_features) > 0:
                # FDR correction if requested
                if self.use_fdr and len(mw_pvalues) > 1:
                    from statsmodels.stats.multitest import multipletests
                    try:
                        reject, pvals_corrected, _, _ = multipletests(
                            mw_pvalues, alpha=self.alpha, method="fdr_bh"
                        )
                        selected_features = [tested_features[i] for i in range(len(tested_features)) 
                                           if reject[i]]
                    except:
                        selected_features = [tested_features[i] for i in range(len(mw_pvalues)) 
                                           if mw_pvalues[i] < self.alpha]
                else:
                    selected_features = [tested_features[i] for i in range(len(mw_pvalues)) 
                                       if mw_pvalues[i] < self.alpha]
                
                # Limit to max_features
                if len(selected_features) > self.max_features:
                    # Sort by p-value and take top
                    p_df = pd.DataFrame({
                        'feature': selected_features, 
                        'p': [mw_pvalues[tested_features.index(f)] for f in selected_features]
                    })
                    p_df = p_df.sort_values('p')
                    self.selected_features_ = p_df.head(self.max_features)['feature'].tolist()
                else:
                    self.selected_features_ = selected_features
                
                self.feature_scores_ = [-np.log10(p+1e-10) for p in mw_pvalues]
        
        # Remove highly correlated features
        if self.selected_features_ is not None and len(self.selected_features_) > 10:
            self.selected_features_ = self._remove_correlated_features(
                X_df.iloc[:, self.selected_features_], 
                self.selected_features_
            )
        
        # Fallback if no features selected
        if self.selected_features_ is None or len(self.selected_features_) < 5:
            logger.warning(f"Few features selected ({len(self.selected_features_) if self.selected_features_ else 0}), using top variance")
            variances = X_df.var()
            self.selected_features_ = variances.nlargest(min(50, X_df.shape[1])).index.tolist()
            self.feature_scores_ = variances.values
        
        return self
    
    def _remove_correlated_features(self, X_subset, feature_indices):
        """Remove highly correlated features to reduce redundancy"""
        corr_matrix = np.abs(np.corrcoef(X_subset.T))
        np.fill_diagonal(corr_matrix, 0)
        
        to_keep = []
        for i, idx in enumerate(feature_indices):
            if not to_keep:
                to_keep.append(idx)
            else:
                # Check correlation with already selected features
                max_corr = 0
                if to_keep:
                    # Get indices in current subset
                    kept_in_subset = [feature_indices.index(k) for k in to_keep]
                    max_corr = np.max(corr_matrix[i, kept_in_subset])
                
                if max_corr < self.correlation_threshold:
                    to_keep.append(idx)
        
        return to_keep[:self.max_features]
    
    def transform(self, X, y=None):
        """Return selected features"""
        if self.selected_features_ is None:
            raise ValueError("Selector not fitted yet")
        return X[:, self.selected_features_]

def bootstrap_ci(y_true, y_score, n_bootstraps=1000, ci=95, random_state=42):
    """
    Calculate bootstrap confidence interval for ROC-AUC
    """
    rng = np.random.RandomState(random_state)
    bootstrapped_scores = []
    
    for i in range(n_bootstraps):
        indices = rng.randint(0, len(y_true), len(y_true))
        if len(np.unique(y_true[indices])) < 2:
            continue
        score = roc_auc_score(y_true[indices], y_score[indices])
        bootstrapped_scores.append(score)
    
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    
    ci_lower = np.percentile(sorted_scores, lower_percentile)
    ci_upper = np.percentile(sorted_scores, upper_percentile)
    
    return ci_lower, ci_upper

def permutation_test_importance(X, y, selected_features, n_permutations=100, random_state=42):
    """
    Test if selected features are better than random
    """
    rng = np.random.RandomState(random_state)
    
    # Score with real features
    X_selected = X[:, selected_features]
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, y, test_size=0.2, stratify=y, random_state=random_state
    )
    
    model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        random_state=random_state,
        eval_metric='logloss',
        use_label_encoder=False,
        n_jobs=-1,
        verbosity=0
    )
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    real_score = roc_auc_score(y_test, y_pred_proba)
    
    # Score with random features
    null_scores = []
    for _ in range(n_permutations):
        random_features = rng.choice(X.shape[1], size=len(selected_features), replace=False)
        X_random = X[:, random_features]
        X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(
            X_random, y, test_size=0.2, stratify=y, random_state=rng.randint(10000)
        )
        model_rand = xgb.XGBClassifier(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.1,
            random_state=random_state,
            eval_metric='logloss',
            use_label_encoder=False,
            n_jobs=-1,
            verbosity=0
        )
        model_rand.fit(X_train_rand, y_train_rand)
        y_pred_rand = model_rand.predict_proba(X_test_rand)[:, 1]
        null_scores.append(roc_auc_score(y_test_rand, y_pred_rand))
    
    p_value = (np.sum(np.array(null_scores) >= real_score) + 1) / (n_permutations + 1)
    return real_score, null_scores, p_value

In [44]:
# ============================================================================
# 1. DATA LOADING
# ============================================================================
print("\n[1/9] Loading data...")

import urllib.request

# List of files to download with their URLs
files_to_download = [
    ("superkingdom2descendents.txt.gz", "https://gmrepo.humangut.info/Downloads/SQLDumps/superkingdom2descendents.txt.gz", "taxonomy table"),
    ("samples_loaded.txt.gz", "https://gmrepo.humangut.info/Downloads/SQLDumps/samples_loaded.txt.gz", "sample table"),
    ("sample_to_run_info.txt.gz", "https://gmrepo.humangut.info/Downloads/SQLDumps/sample_to_run_info.txt.gz", "sample run"),
    ("species_abundance.txt.gz", "https://gmrepo.humangut.info/Downloads/SQLDumps/species_abundance.txt.gz", "abundance table")
]

for filename, url, description in files_to_download:
    if not os.path.exists(filename):
        print(f"Downloading {description}...")
        urllib.request.urlretrieve(url, filename)
        print(f"{description} download finished")
    else:
        print(f"{filename} already exists, skipping download")

# Decompress files
import gzip
import shutil

def decompress_gz(gz_file):
    """Decompress a .gz file"""
    output_file = gz_file.replace('.gz', '')
    if not os.path.exists(output_file):
        with gzip.open(gz_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Decompressed: {gz_file} -> {output_file}")
    else:
        print(f"{output_file} already exists, skipping decompression")

files_to_decompress = [
    "superkingdom2descendents.txt.gz",
    "samples_loaded.txt.gz",
    "sample_to_run_info.txt.gz",
    "species_abundance.txt.gz"
]

for file in files_to_decompress:
    decompress_gz(file)

# Load data
abundance_raw = pd.read_table("species_abundance.txt")
taxonomy_table = pd.read_table("superkingdom2descendents.txt")
sample_table = pd.read_table("samples_loaded.txt")
run_table = pd.read_table("sample_to_run_info.txt", dtype='str')

logger.info(f"Loaded {len(abundance_raw)} abundance records")
logger.info(f"Loaded {len(taxonomy_table)} taxonomy entries")
logger.info(f"Loaded {len(sample_table)} samples")


[1/9] Loading data...
superkingdom2descendents.txt.gz already exists, skipping download
samples_loaded.txt.gz already exists, skipping download
sample_to_run_info.txt.gz already exists, skipping download
species_abundance.txt.gz already exists, skipping download
superkingdom2descendents.txt already exists, skipping decompression
samples_loaded.txt already exists, skipping decompression
sample_to_run_info.txt already exists, skipping decompression
species_abundance.txt already exists, skipping decompression


2026-02-05 16:11:33,214 - INFO - Loaded 5541271 abundance records
2026-02-05 16:11:33,214 - INFO - Loaded 5195 taxonomy entries
2026-02-05 16:11:33,214 - INFO - Loaded 108176 samples


In [45]:
# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================
print("\n[2/9] Preprocessing abundance data...")

# Merge abundance with taxonomy
abundance_with_taxonomy = pd.merge(
    abundance_raw,
    taxonomy_table[['ncbi_taxon_id', 'superkingdom']].drop_duplicates(),
    on='ncbi_taxon_id',
    how='left'
)

abundance_with_taxonomy['superkingdom'] = abundance_with_taxonomy['superkingdom'].fillna('Unclassified')

# Filter to Bacteria and Archaea
abundance_raw = abundance_with_taxonomy[
    (abundance_with_taxonomy['superkingdom'] == 'Bacteria') | 
    (abundance_with_taxonomy['superkingdom'] == 'Archaea') |
    (abundance_with_taxonomy['ncbi_taxon_id'] == -1)
].copy()

abundance_raw = abundance_raw.drop(columns=['superkingdom'])

logger.info(f"Filtered abundance data shape: {abundance_raw.shape}")

# Filter to genus level
abundance_genus = abundance_raw[abundance_raw['taxon_rank_level'].str.contains('genus', case=False)]

# Pivot table: samples as rows, taxa as columns
pivoted_df = abundance_genus.pivot_table(
    index='loaded_uid', 
    columns='ncbi_taxon_id', 
    values='relative_abundance', 
    fill_value=0
)

# Rename columns with genus names
taxonomy_table['scientific_name'] = taxonomy_table['scientific_name'].str.replace(' ', '_')
mapping = dict(zip(taxonomy_table['ncbi_taxon_id'], taxonomy_table['scientific_name']))
pivoted_df.rename(columns=mapping, inplace=True)

# Clean column names
pivoted_df.columns = pivoted_df.columns.str.replace(r'[\[\]<>]', '_', regex=True)

logger.info(f"Created abundance matrix: {pivoted_df.shape}")


[2/9] Preprocessing abundance data...


2026-02-05 16:11:34,382 - INFO - Filtered abundance data shape: (5534776, 6)
2026-02-05 16:11:37,214 - INFO - Created abundance matrix: (68723, 2122)


In [46]:
# ============================================================================
# 3. METADATA CLEANING
# ============================================================================
print("\n[3/9] Cleaning metadata...")

# Merge metadata tables
metadata_df = pd.merge(
    run_table,
    sample_table,
    left_on='run_id',
    right_on='accession_id',
    how='inner'
)

# Convert nr_reads_sequenced to numeric (for histogram)
metadata_df['nr_reads_sequenced_numeric'] = pd.to_numeric(metadata_df['nr_reads_sequenced'], errors='coerce')

# Convert QCStatus to categorical
metadata_df['QCStatus_cat'] = metadata_df['QCStatus'].astype('category')

# Convert nr_reads_sequenced to numeric
metadata_df['nr_reads_sequenced'] = pd.to_numeric(metadata_df['nr_reads_sequenced'], errors='coerce')

# Apply filters
metadata_df = metadata_df[
    (metadata_df['data_type'].str.lower().isin(['amplicon', 'amplicon sequencing'])) &
    (metadata_df['tool_used'].str.lower() == 'qiime') &
    (metadata_df['QCStatus_cat'] == 1) &
    (metadata_df['Recent_Antibiotics_Use'].str.lower() != 'yes') &
    (metadata_df['nr_reads_sequenced'] > 50000)
].copy()

logger.info(f"Samples after full filtering: {len(metadata_df)}")

# ------------------------------------------------------------------
# DEFINE BATCH VARIABLE (not used, but left just in case)
# ------------------------------------------------------------------
# Prefer run_id; fallback to project_id if needed
metadata_df['batch'] = metadata_df['project_id']
metadata_df['batch'] = metadata_df['batch'].astype(str)
logger.info(f"Number of batches: {metadata_df['batch'].nunique()}")

# Quality filters
metadata_df = metadata_df[metadata_df['QCStatus'] != 0]

# Remove phenotypes with < 100 samples
phenotype_counts = metadata_df['phenotype'].value_counts()
phenotypes_to_keep = phenotype_counts[phenotype_counts >= 100].index
metadata_df = metadata_df[metadata_df['phenotype'].isin(phenotypes_to_keep)]

logger.info(f"Kept {len(phenotypes_to_keep)} phenotypes with ≥100 samples")

# Standardize phenotype names
metadata_df['phenotype'] = metadata_df['phenotype'].replace(
    ['healthy', 'Health', 'Normal'], 'Healthy'
)
metadata_df['phenotype'] = metadata_df['phenotype'].replace(
    ['IBD', 'Inflamatory Bowel Diseases'], 'Inflammatory Bowel Disease'
)

# Remove duplicates
non_phenotype_columns = metadata_df.columns.difference(['phenotype'])
duplicated_rows = metadata_df.duplicated(subset=non_phenotype_columns, keep=False)
metadata_df = metadata_df[~duplicated_rows]

logger.info(f"Removed {duplicated_rows.sum()} duplicate rows")

2026-02-05 16:11:37,412 - INFO - Samples after full filtering: 24970
2026-02-05 16:11:37,414 - INFO - Number of batches: 415
2026-02-05 16:11:37,437 - INFO - Kept 36 phenotypes with ≥100 samples



[3/9] Cleaning metadata...


2026-02-05 16:11:37,475 - INFO - Removed 0 duplicate rows


In [47]:
# ============================================================================
# 4. ABUNDANCE DATA CLEANING
# ============================================================================
print("\n[4/9] Cleaning abundance data...")

# Keep only samples in metadata
uids_to_keep = metadata_df["uid"]
pivoted_df_filtered = pivoted_df.loc[pivoted_df.index.isin(uids_to_keep)]

# Remove unknown column (first column if it exists)
if pivoted_df_filtered.columns[0] == -1 or 'unknown' in str(pivoted_df_filtered.columns[0]).lower():
    pivoted_df_filtered = pivoted_df_filtered.iloc[:, 1:]

# Verify alignment
metadata_df = metadata_df.set_index('uid').loc[pivoted_df_filtered.index].reset_index()
pivoted_df_filtered = pivoted_df_filtered.reset_index(drop=True)
metadata_df = metadata_df.reset_index(drop=True)

logger.info(f"Final dataset: {len(pivoted_df_filtered)} samples × {len(pivoted_df_filtered.columns)} features")

2026-02-05 16:11:37,610 - INFO - Final dataset: 18939 samples × 2122 features



[4/9] Cleaning abundance data...


In [48]:
# ============================================================================
# 5. HEALTHY SAMPLES - OUTLIER REMOVAL
# ============================================================================
print("\n[5/9] Processing healthy samples and removing outliers...")

# Subset healthy samples
healthy_metadata = metadata_df[metadata_df["phenotype"] == "Healthy"].copy()
pivoted_df_Healthy = pivoted_df_filtered.loc[healthy_metadata.index].copy()

# Reset indices
pivoted_df_Healthy = pivoted_df_Healthy.reset_index(drop=True)
healthy_metadata = healthy_metadata.reset_index(drop=True)

# Apply CLR transformation
logger.info("Applying CLR transformation to healthy samples...")
healthy_clr = clr_transform(pivoted_df_Healthy.values)

# Standardize for PCA
scaler_healthy = StandardScaler()
healthy_standardized = scaler_healthy.fit_transform(healthy_clr)

# PCA for outlier detection
pca = PCA(n_components=2, random_state=RANDOM_SEED)
pca_result = pca.fit_transform(healthy_standardized)
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])

# Identify outliers using Z-scores
z_scores = zscore(pca_df)
outlier_threshold = 3
outliers = (np.abs(z_scores) > outlier_threshold).any(axis=1)

logger.info(f"Identified {outliers.sum()} outliers in healthy samples")

# Remove outliers
pivoted_df_Healthy = pivoted_df_Healthy[~outliers].reset_index(drop=True)
healthy_metadata = healthy_metadata[~outliers].reset_index(drop=True)

logger.info(f"Clean healthy samples: {len(pivoted_df_Healthy)}")

# ============================================================================
# 5b. PREPARE NON-HEALTHY SAMPLES DATA
# ============================================================================
print("\n[5b/9] Preparing non-healthy samples data...")

non_healthy_metadata = metadata_df[metadata_df["phenotype"] != "Healthy"].reset_index(drop=True)
pivoted_df_non_Healthy = pivoted_df_filtered.loc[
    metadata_df[metadata_df["phenotype"] != "Healthy"].index
].reset_index(drop=True)

logger.info(f"Non-healthy samples: {len(pivoted_df_non_Healthy)}")
logger.info(f"Number of diseases: {non_healthy_metadata['phenotype'].nunique()}")


[5/9] Processing healthy samples and removing outliers...


2026-02-05 16:11:37,698 - INFO - Applying CLR transformation to healthy samples...
2026-02-05 16:11:38,602 - INFO - Identified 55 outliers in healthy samples
2026-02-05 16:11:38,653 - INFO - Clean healthy samples: 9756
2026-02-05 16:11:38,710 - INFO - Non-healthy samples: 9128
2026-02-05 16:11:38,711 - INFO - Number of diseases: 35



[5b/9] Preparing non-healthy samples data...


In [49]:
# ============================================================================
# 6. VISUALIZATION - HEALTHY SAMPLES (UMAP)
# ============================================================================
print("\n[6/9] Creating visualizations...")

# Re-apply CLR transformation after outlier removal
healthy_clr_clean = clr_transform(pivoted_df_Healthy.values)
scaler_viz = StandardScaler()
healthy_standardized_clean = scaler_viz.fit_transform(healthy_clr_clean)

# UMAP
logger.info("Computing UMAP for healthy samples...")
umap_reducer = umap.UMAP(n_components=2, random_state=RANDOM_SEED, n_neighbors=15, min_dist=0.1)
umap_result = umap_reducer.fit_transform(healthy_standardized_clean)

# Plot
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.scatter(umap_result[:, 0], umap_result[:, 1], alpha=0.5, s=10, color='steelblue')
ax.set_title('UMAP of Healthy Samples (Outliers Removed)', fontsize=14, fontweight='bold')
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')  # FIXED: Changed set_yabel to set_ylabel
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('healthy_samples_umap.png', dpi=300, bbox_inches='tight')
logger.info("Saved: healthy_samples_umap.png")
plt.close()

# ============================================================================
# 6b. VISUALIZATION - COMBINED HEALTHY VS DISEASES
# ============================================================================
print("\n[6b/9] Creating combined visualization (subsampled for speed)...")

# Subsample for visualization to avoid memory issues
max_viz_samples = 1000000
if len(pivoted_df_Healthy) + len(pivoted_df_non_Healthy) > max_viz_samples:
    # Subsample proportionally
    n_healthy_viz = min(len(pivoted_df_Healthy), max_viz_samples // 2)
    n_disease_viz = min(len(pivoted_df_non_Healthy), max_viz_samples // 2)
    
    rng_viz = np.random.RandomState(RANDOM_SEED)
    healthy_viz_idx = rng_viz.choice(len(pivoted_df_Healthy), n_healthy_viz, replace=False)
    disease_viz_idx = rng_viz.choice(len(pivoted_df_non_Healthy), n_disease_viz, replace=False)
    
    healthy_viz = pivoted_df_Healthy.iloc[healthy_viz_idx]
    disease_viz = pivoted_df_non_Healthy.iloc[disease_viz_idx]
    disease_labels_viz = non_healthy_metadata.iloc[disease_viz_idx]['phenotype']
    
    logger.info(f"Subsampled to {n_healthy_viz} healthy + {n_disease_viz} disease samples for visualization")
else:
    healthy_viz = pivoted_df_Healthy
    disease_viz = pivoted_df_non_Healthy
    disease_labels_viz = non_healthy_metadata['phenotype']

# CLR transform combined data
combined_viz = pd.concat([healthy_viz, disease_viz], axis=0).reset_index(drop=True)
combined_viz_clr = clr_transform(combined_viz.values)
combined_viz_scaled = StandardScaler().fit_transform(combined_viz_clr)

# UMAP
logger.info("Computing UMAP for combined data...")
umap_combined = umap.UMAP(n_components=2, random_state=RANDOM_SEED, n_neighbors=15, min_dist=0.1)
umap_combined_result = umap_combined.fit_transform(combined_viz_scaled)

# Create labels
combined_labels = ['Healthy'] * len(healthy_viz) + disease_labels_viz.tolist()

# Get unique diseases
unique_diseases = sorted(non_healthy_metadata['phenotype'].unique())
np.random.seed(RANDOM_SEED)
disease_colors = {}
for disease in unique_diseases:
    disease_colors[disease] = (np.random.random(), np.random.random(), np.random.random())

# Plot
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Plot healthy in background
healthy_mask = np.array([l == 'Healthy' for l in combined_labels])
ax.scatter(umap_combined_result[healthy_mask, 0], umap_combined_result[healthy_mask, 1],
          alpha=0.3, s=8, color='lightgray', label='Healthy', zorder=1)

# Plot diseases on top
for disease in unique_diseases:
    disease_mask = np.array([l == disease for l in combined_labels])
    if disease_mask.sum() > 0:
        ax.scatter(umap_combined_result[disease_mask, 0], umap_combined_result[disease_mask, 1],
                  alpha=0.7, s=20, color=disease_colors[disease], label=disease, zorder=2)

ax.set_title('UMAP: Healthy vs All Diseases', fontsize=14, fontweight='bold')
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.grid(alpha=0.3)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=7, ncol=1, framealpha=0.9)

plt.tight_layout()
plt.savefig('combined_healthy_disease_umap.png', dpi=300, bbox_inches='tight')
logger.info("Saved: combined_healthy_disease_umap.png")
plt.close()


[6/9] Creating visualizations...


2026-02-05 16:11:39,076 - INFO - Computing UMAP for healthy samples...
2026-02-05 16:12:01,411 - INFO - Saved: healthy_samples_umap.png



[6b/9] Creating combined visualization (subsampled for speed)...


2026-02-05 16:12:02,163 - INFO - Computing UMAP for combined data...
2026-02-05 16:12:28,955 - INFO - Saved: combined_healthy_disease_umap.png


In [50]:
# ============================================================================
# 7. DISEASE CLASSIFICATION LOOP - WITH NESTED CV AND IMPROVED FEATURE SELECTION
# ============================================================================
print("\n[7/9] Training disease classifiers with nested CV and improved feature selection...")

all_results = []
failed_diseases = []
feature_importance_dict = {}
shap_explanations = {}  # NEW: Store SHAP explanations

diseases = non_healthy_metadata["phenotype"].unique()
logger.info(f"Processing {len(diseases)} diseases...")

for disease in tqdm(diseases, desc="Disease classification"):
    
    logger.info(f"\n{'='*80}")
    logger.info(f"Processing: {disease}")
    logger.info(f"{'='*80}")
    
    try:
        # 7.1 Subset disease samples
        disease_mask = non_healthy_metadata["phenotype"] == disease
        disease_metadata_subset = non_healthy_metadata[disease_mask].reset_index(drop=True)
        disease_data = pivoted_df_non_Healthy[disease_mask].reset_index(drop=True)
        
        n_disease = len(disease_data)
        n_healthy_total = len(pivoted_df_Healthy)
        
        logger.info(f"Disease samples: {n_disease}")
        logger.info(f"Healthy samples available: {n_healthy_total}")
        
        if n_disease < 20:
            logger.warning(f"Skipping {disease}: too few disease samples (<20)")
            failed_diseases.append((disease, "Too few samples"))
            continue
        
        # 7.2 Balance classes
        healthy_sample_size = min(n_disease * 2, n_healthy_total)
        rng = np.random.RandomState(RANDOM_SEED)
        healthy_idx = rng.choice(n_healthy_total, size=healthy_sample_size, replace=False)
        
        balanced_healthy_data = pivoted_df_Healthy.iloc[healthy_idx].reset_index(drop=True)
        
        logger.info(f"Balanced healthy samples: {len(balanced_healthy_data)} (no replacement)")
        
        # 7.3 Combine data and apply CLR transformation
        combined_data = pd.concat([balanced_healthy_data, disease_data], axis=0).reset_index(drop=True)
        combined_labels = np.array([0] * len(balanced_healthy_data) + [1] * len(disease_data))
        
        # CLR transformation on combined data
        logger.info("Applying CLR transformation...")
        X = clr_transform(combined_data.values)
        y = combined_labels
        
        # Get feature names for SHAP
        feature_names = combined_data.columns.tolist()
        
        logger.info(f"Dataset shape before split: {X.shape}")
        logger.info(f"Class distribution: Healthy={(y==0).sum()}, {disease}={(y==1).sum()}")
        
        # 7.4 NESTED CROSS-VALIDATION SETUP
        logger.info("Setting up nested cross-validation...")
        
        # Inner CV for feature selection and hyperparameter tuning
        inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)
        
        # Outer CV for performance evaluation
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
        
        # ====================================================================
        # NEW: Create pipeline with feature selection
        # ====================================================================
        from sklearn.pipeline import Pipeline
        
        # Create pipeline with improved feature selector
        pipeline = Pipeline([
            ('feature_selector', CompositionalFeatureSelector(
                method='mutual_info',  # Can change to 'mannwhitney'
                max_features=500,
                alpha=0.05,
                correlation_threshold=0.9
            )),
            ('scaler', StandardScaler()),
            ('classifier', xgb.XGBClassifier(
                random_state=RANDOM_SEED,
                eval_metric='logloss',
                use_label_encoder=False,
                verbosity=0
            ))
        ])
        
        # Hyperparameter grid for inner CV
        param_grid = {
            'classifier__n_estimators': [100],
            'classifier__max_depth': [4],
            'classifier__learning_rate': [0.1],
            'feature_selector__max_features': [500],
            'feature_selector__method': ['mutual_info', 'mannwhitney']
        }
        
        # 7.5 NESTED CROSS-VALIDATION
        logger.info("Running nested cross-validation...")
        
        # Grid search with inner CV
        grid_search = GridSearchCV(
            pipeline,
            param_grid,
            cv=inner_cv,
            scoring='roc_auc',
            n_jobs=-1,
            verbose=0
        )
        
        # Outer CV for unbiased evaluation
        outer_cv_results = cross_validate(
            grid_search,
            X,
            y,
            cv=outer_cv,
            scoring=['roc_auc', 'f1', 'precision', 'recall', 'accuracy'],
            return_train_score=True,
            return_estimator=True,
            n_jobs=-1
        )
        
        # 7.6 Train final model on all data with best parameters
        logger.info("Training final model with best parameters...")
        
        # Fit grid search on all data to get best estimator
        grid_search.fit(X, y)
        best_model = grid_search.best_estimator_
        
        # Get feature selector from best model
        feature_selector = best_model.named_steps['feature_selector']
        selected_features = feature_selector.selected_features_
        
        # Get selected feature names
        selected_feature_names = [feature_names[i] for i in selected_features]
        
        # DEBUG: Print selected features info
        logger.info(f"DEBUG: Selected {len(selected_features)} features for {disease}")
        if len(selected_features) > 0:
            logger.info(f"DEBUG: First 5 selected feature indices: {selected_features[:5]}")
            logger.info(f"DEBUG: First 5 selected feature names: {selected_feature_names[:5]}")
        
        # 7.7 Create train/test split for additional validation
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED
        )
        
        # Fit best model on training data
        best_model.fit(X_train, y_train)
        
        # After fitting, get the updated feature selector from the fitted model
        fitted_feature_selector = best_model.named_steps['feature_selector']
        fitted_selected_features = fitted_feature_selector.selected_features_
        fitted_selected_feature_names = [feature_names[i] for i in fitted_selected_features]
        
        # Get classifier from fitted model
        fitted_classifier = best_model.named_steps['classifier']
        
        # DEBUG: Print fitted features info
        logger.info(f"DEBUG: After fitting, selected {len(fitted_selected_features)} features")
        logger.info(f"DEBUG: Feature importances array length: {len(fitted_classifier.feature_importances_)}")
        
        # Predictions on test set
        y_test_pred = best_model.predict(X_test)
        y_test_proba = best_model.predict_proba(X_test)[:, 1]
        
        # Test set metrics
        test_roc_auc = roc_auc_score(y_test, y_test_proba)
        test_f1 = f1_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred, zero_division=0)
        test_recall = recall_score(y_test, y_test_pred, zero_division=0)
        
        # Bootstrap confidence intervals for test set
        ci_lower, ci_upper = bootstrap_ci(y_test, y_test_proba, n_bootstraps=1000, random_state=RANDOM_SEED)
        
        # 7.8 Baseline model (dummy classifier)
        logger.info("Training baseline dummy classifier...")
        
        # Use same feature selection for fair comparison
        X_train_selected = fitted_feature_selector.transform(X_train)
        X_test_selected = fitted_feature_selector.transform(X_test)
        
        # Scale
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)
        
        dummy = DummyClassifier(strategy='stratified', random_state=RANDOM_SEED)
        dummy.fit(X_train_scaled, y_train)
        y_dummy_proba = dummy.predict_proba(X_test_scaled)[:, 1]
        dummy_roc_auc = roc_auc_score(y_test, y_dummy_proba)
        
        # 7.9 Store results
        cv_roc_mean = float(np.mean(outer_cv_results['test_roc_auc']))
        cv_roc_std = float(np.std(outer_cv_results['test_roc_auc']))
        cv_f1_mean = float(np.mean(outer_cv_results['test_f1']))
        cv_f1_std = float(np.std(outer_cv_results['test_f1']))
        
        train_roc_mean = float(np.mean(outer_cv_results['train_roc_auc']))
        train_f1_mean = float(np.mean(outer_cv_results['train_f1']))
        
        # Calculate overfitting gap from outer CV
        overfit_gap = train_roc_mean - cv_roc_mean
        
        all_results.append({
            'Disease': disease,
            'N_Disease': n_disease,
            'N_Healthy_Balanced': len(balanced_healthy_data),
            'N_Features_Selected': len(fitted_selected_features),
            
            # Nested CV metrics (unbiased)
            'CV_ROC_AUC_Mean': cv_roc_mean,
            'CV_ROC_AUC_Std': cv_roc_std,
            'CV_F1_Mean': cv_f1_mean,
            'CV_F1_Std': cv_f1_std,
            'CV_Precision_Mean': float(np.mean(outer_cv_results['test_precision'])),
            'CV_Recall_Mean': float(np.mean(outer_cv_results['test_recall'])),
            
            # Training set performance (from outer CV)
            'Train_ROC_AUC_Mean': train_roc_mean,
            'Train_F1_Mean': train_f1_mean,
            
            # Test set performance
            'Test_ROC_AUC': float(test_roc_auc),
            'Test_ROC_AUC_CI_Lower': float(ci_lower),
            'Test_ROC_AUC_CI_Upper': float(ci_upper),
            'Test_F1': float(test_f1),
            'Test_Precision': float(test_precision),
            'Test_Recall': float(test_recall),
            
            # Baseline
            'Dummy_ROC_AUC': float(dummy_roc_auc),
            
            # Model info
            'Best_Params': str(grid_search.best_params_),
            'Feature_Selector_Method': fitted_feature_selector.method,
            
            # Overfitting indicator from nested CV
            'Overfit_Gap': float(overfit_gap)
        })
        
        # Store feature importance - Use the fitted model's components
        if hasattr(fitted_classifier, 'feature_importances_'):
            # Ensure arrays have the same length
            if len(fitted_selected_features) == len(fitted_classifier.feature_importances_):
                feature_df = pd.DataFrame({
                    'feature_index': fitted_selected_features,
                    'feature_name': fitted_selected_feature_names,
                    'importance': fitted_classifier.feature_importances_
                }).sort_values('importance', ascending=False)
                
                feature_importance_dict[disease] = feature_df
                
                # Save feature importances to CSV
                safe_name = str(disease).replace("/", "_").replace(" ", "_")
                feature_csv_filename = f'feature_importances_{safe_name}.csv'
                feature_df.to_csv(feature_csv_filename, index=False)
                logger.info(f"✓ Saved feature importances: {feature_csv_filename}")
                
                # Print top features for debugging
                top_features = feature_df.head(10)
                logger.info(f"DEBUG: Top 10 features for {disease}:")
                for idx, row in top_features.iterrows():
                    logger.info(f"  - {row['feature_name']}: {row['importance']:.4f}")
            else:
                logger.warning(f"Mismatch in array lengths for {disease}: "
                              f"selected_features={len(fitted_selected_features)}, "
                              f"importances={len(fitted_classifier.feature_importances_)}")
                # Fallback: use what we have with proper alignment
                n_features = min(len(fitted_selected_features), len(fitted_classifier.feature_importances_))
                feature_df = pd.DataFrame({
                    'feature_index': fitted_selected_features[:n_features],
                    'feature_name': fitted_selected_feature_names[:n_features],
                    'importance': fitted_classifier.feature_importances_[:n_features]
                }).sort_values('importance', ascending=False)
                
                feature_importance_dict[disease] = feature_df
                
                # Save feature importances to CSV
                safe_name = str(disease).replace("/", "_").replace(" ", "_")
                feature_csv_filename = f'feature_importances_{safe_name}.csv'
                feature_df.to_csv(feature_csv_filename, index=False)
                logger.info(f"✓ Saved feature importances (partial): {feature_csv_filename}")
        
        logger.info(f"✓ Nested CV ROC-AUC: {cv_roc_mean:.4f} ± {cv_roc_std:.4f}")
        logger.info(f"✓ Test ROC-AUC: {test_roc_auc:.4f} (95% CI: [{ci_lower:.4f}, {ci_upper:.4f}])")
        logger.info(f"✓ Baseline (dummy): {dummy_roc_auc:.4f}")
        logger.info(f"✓ Best params: {grid_search.best_params_}")
        logger.info(f"✓ Selected features: {len(fitted_selected_features)}")
        
        # ====================================================================
        # NEW: SHAP INTERPRETABILITY - FIXED VERSION
        # ====================================================================
        if test_roc_auc > 0.65 and len(fitted_selected_feature_names) > 0:  # Only create SHAP for decent models and if we have features
            try:
                logger.info("Computing SHAP values for model interpretability...")
                
                # Check if SHAP is installed
                try:
                    import shap
                    SHAP_AVAILABLE = True
                except ImportError:
                    logger.warning("SHAP not installed. Install with: pip install shap")
                    SHAP_AVAILABLE = False
                    shap = None
                
                if SHAP_AVAILABLE:
                    # Prepare data for SHAP
                    X_test_selected = fitted_feature_selector.transform(X_test)
                    X_test_scaled = best_model.named_steps['scaler'].transform(X_test_selected)
                    
                    # Create SHAP explainer
                    explainer = shap.TreeExplainer(fitted_classifier)
                    
                    # Calculate SHAP values
                    shap_values = explainer.shap_values(X_test_scaled)
                    
                    # Store SHAP explanations
                    shap_explanations[disease] = {
                        'shap_values': shap_values,
                        'expected_value': explainer.expected_value,
                        'feature_names': fitted_selected_feature_names,
                        'X_test': X_test_scaled
                    }
                    
                    # Check dimensions for debugging
                    logger.info(f"DEBUG: SHAP values shape: {shap_values.shape}")
                    logger.info(f"DEBUG: Number of features in SHAP: {shap_values.shape[1]}")
                    logger.info(f"DEBUG: Number of selected feature names: {len(fitted_selected_feature_names)}")
                    
                    safe_name = str(disease).replace("/", "_").replace(" ", "_")
                    
                    # Create SHAP summary plot - FIXED: Pass ALL feature names, not just first 20
                    try:
                        plt.figure(figsize=(12, 8))
                        n_features_to_display = min(20, len(fitted_selected_feature_names), shap_values.shape[1])
                        shap.summary_plot(
                            shap_values, 
                            X_test_scaled,
                            feature_names=fitted_selected_feature_names,  # Pass ALL feature names
                            max_display=n_features_to_display,
                            show=False
                        )
                        plt.title(f'SHAP Summary Plot for {disease}\nTest AUC: {test_roc_auc:.3f}', 
                                 fontsize=14, fontweight='bold')
                        plt.tight_layout()
                        plt.savefig(f'shap_summary_{safe_name}.png', dpi=300, bbox_inches='tight')
                        logger.info(f"✓ Saved SHAP summary: shap_summary_{safe_name}.png")
                        plt.close()
                    except Exception as summary_e:
                        logger.error(f"Failed to create SHAP summary plot: {summary_e}")
                    
                    # Create SHAP bar plot - FIXED: Pass ALL feature names
                    try:
                        plt.figure(figsize=(10, 6))
                        shap.summary_plot(
                            shap_values, 
                            X_test_scaled,
                            feature_names=fitted_selected_feature_names,  # Pass ALL feature names
                            plot_type='bar',
                            show=False
                        )
                        plt.title(f'Feature Importance for {disease}', fontsize=14, fontweight='bold')
                        plt.tight_layout()
                        plt.savefig(f'shap_bar_{safe_name}.png', dpi=300, bbox_inches='tight')
                        logger.info(f"✓ Saved SHAP bar: shap_bar_{safe_name}.png")
                        plt.close()
                    except Exception as bar_e:
                        logger.error(f"Failed to create SHAP bar plot: {bar_e}")
                    
                    # Create SHAP dependence plot for top feature - FIXED VERSION
                    if len(fitted_selected_feature_names) > 0 and shap_values.shape[1] > 0:
                        try:
                            # Calculate mean absolute SHAP values for each feature
                            mean_abs_shap = np.abs(shap_values).mean(0)
                            top_feature_idx = int(np.argmax(mean_abs_shap))
                            
                            logger.info(f"DEBUG: Top feature index: {top_feature_idx}")
                            logger.info(f"DEBUG: Mean absolute SHAP shape: {mean_abs_shap.shape}")
                            
                            # Double-check bounds
                            if top_feature_idx < len(fitted_selected_feature_names) and top_feature_idx < shap_values.shape[1]:
                                top_feature_name = fitted_selected_feature_names[top_feature_idx]
                                logger.info(f"DEBUG: Top feature name: {top_feature_name}")
                                
                                plt.figure(figsize=(10, 6))
                                shap.dependence_plot(
                                    top_feature_idx,
                                    shap_values,
                                    X_test_scaled,
                                    feature_names=fitted_selected_feature_names,
                                    show=False
                                )
                                plt.title(f'SHAP Dependence Plot: {top_feature_name}', 
                                         fontsize=14, fontweight='bold')
                                plt.tight_layout()
                                plt.savefig(f'shap_dependence_{safe_name}.png', dpi=300, bbox_inches='tight')
                                logger.info(f"✓ Saved SHAP dependence: shap_dependence_{safe_name}.png")
                                plt.close()
                            else:
                                logger.warning(f"Top feature index {top_feature_idx} out of bounds for feature names (length {len(fitted_selected_feature_names)}) or SHAP values (shape {shap_values.shape})")
                        except Exception as dep_e:
                            logger.warning(f"Could not create dependence plot: {dep_e}")
                    
            except Exception as e:
                logger.error(f"Failed to compute SHAP for {disease}: {e}")
                import traceback
                logger.error(traceback.format_exc())
        
        # 7.10 Create individual disease visualization (3D UMAP) for good models
        if test_roc_auc > 0.70:
            try:
                logger.info("Creating 3D UMAP visualization...")
                
                # Use the scaled training + test data
                X_viz = np.vstack([X_train, X_test])
                y_viz = np.hstack([y_train, y_test])
                
                # UMAP 3D
                umap_3d = umap.UMAP(n_components=3, random_state=RANDOM_SEED, n_neighbors=15, min_dist=0.1)
                umap_3d_result = umap_3d.fit_transform(X_viz)
                
                # Create interactive plot with plotly
                import plotly.graph_objects as go
                
                umap_healthy = umap_3d_result[y_viz == 0]
                umap_disease = umap_3d_result[y_viz == 1]
                
                fig = go.Figure()
                
                fig.add_trace(go.Scatter3d(
                    x=umap_healthy[:, 0],
                    y=umap_healthy[:, 1],
                    z=umap_healthy[:, 2],
                    mode='markers',
                    name='Healthy',
                    marker=dict(size=4, opacity=0.6, color='#2E86AB'),
                ))
                
                fig.add_trace(go.Scatter3d(
                    x=umap_disease[:, 0],
                    y=umap_disease[:, 1],
                    z=umap_disease[:, 2],
                    mode='markers',
                    name=str(disease),
                    marker=dict(size=4, opacity=0.6, color='#A23B72'),
                ))
                
                fig.update_layout(
                    title=f"3D UMAP: Healthy vs {disease}<br>Test AUC: {test_roc_auc:.3f} (95% CI: [{ci_lower:.3f}, {ci_upper:.3f}])",
                    scene=dict(
                        xaxis_title="UMAP 1",
                        yaxis_title="UMAP 2",
                        zaxis_title="UMAP 3",
                    ),
                    legend=dict(itemsizing='constant'),
                    margin=dict(l=0, r=0, b=0, t=60),
                )
                
                safe_name = str(disease).replace("/", "_").replace(" ", "_")
                html_filename = f"{safe_name}_umap_3D.html"
                fig.write_html(html_filename, include_plotlyjs='cdn')
                
                if os.path.exists(html_filename):
                    logger.info(f"✓ Saved 3D UMAP: {html_filename}")
                else:
                    logger.warning(f"Failed to save {html_filename}")
                    
            except Exception as e:
                logger.error(f"Failed to create 3D UMAP for {disease}: {e}")
        
    except Exception as e:
        logger.error(f"Failed to process {disease}: {str(e)}")
        failed_diseases.append((disease, str(e)))
        import traceback
        traceback.print_exc()
        continue

logger.info(f"\n{'='*80}")
logger.info(f"Disease classification loop completed")
logger.info(f"Successfully processed: {len(all_results)} diseases")
logger.info(f"Failed: {len(failed_diseases)} diseases")
logger.info(f"{'='*80}")

if len(failed_diseases) > 0:
    logger.info("\nFailed diseases:")
    for disease, reason in failed_diseases:
        logger.info(f"  - {disease}: {reason}")

2026-02-05 16:12:28,976 - INFO - Processing 35 diseases...



[7/9] Training disease classifiers with nested CV and improved feature selection...


Disease classification:   0%|                            | 0/35 [00:00<?, ?it/s]2026-02-05 16:12:28,979 - INFO - 
2026-02-05 16:12:28,979 - INFO - Processing: Obesity
2026-02-05 16:12:28,980 - INFO - ================================================================================
2026-02-05 16:12:28,982 - INFO - Disease samples: 243
2026-02-05 16:12:28,983 - INFO - Healthy samples available: 9756
2026-02-05 16:12:28,989 - INFO - Balanced healthy samples: 486 (no replacement)
2026-02-05 16:12:28,992 - INFO - Applying CLR transformation...
2026-02-05 16:12:29,008 - INFO - Dataset shape before split: (729, 2122)
2026-02-05 16:12:29,008 - INFO - Class distribution: Healthy=486, Obesity=243
2026-02-05 16:12:29,009 - INFO - Setting up nested cross-validation...
2026-02-05 16:12:29,009 - INFO - Running nested cross-validation...
/home/cesc/miniconda3/envs/microbiome_classif2/lib/python3.14/site-packages/sklearn/utils/parallel.py:144: UserWarning: `sklearn.utils.parallel.delayed` should be use

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [51]:
# ============================================================================
# 8. RESULTS ANALYSIS AND VISUALIZATION
# ============================================================================
print("\n[8/9] Analyzing results...")

results_df = pd.DataFrame(all_results)

if len(results_df) == 0:
    logger.error("No results to analyze!")
    print("\n" + "="*80)
    print("ANALYSIS FAILED - NO SUCCESSFUL CLASSIFICATIONS")
    print("="*80)
else:
    # Save all results
    results_df.to_csv('xgboost_classification_results_nested_cv.csv', index=False)
    logger.info("✓ Saved: xgboost_classification_results_nested_cv.csv")
    
    # Sort by nested CV performance (most reliable)
    results_df = results_df.sort_values('CV_ROC_AUC_Mean', ascending=False)
    
    # Define good models based on nested CV performance
    good_models = results_df[
        (results_df['CV_ROC_AUC_Mean'] > 0.65) &  # Realistic threshold for nested CV
        (results_df['Overfit_Gap'] < 0.15)  # Not severely overfitting
    ].copy()
    
    logger.info(f"\n✓ Found {len(good_models)} diseases with good nested CV performance")
    logger.info(f"  (CV AUC > 0.65, Overfitting gap < 0.15)")
    
    # Summary statistics
    print("\n" + "="*80)
    print("PERFORMANCE SUMMARY (NESTED CV)")
    print("="*80)
    print(f"Total diseases analyzed: {len(results_df)}")
    print(f"Diseases with CV ROC-AUC > 0.65: {len(results_df[results_df['CV_ROC_AUC_Mean'] > 0.65])}")
    print(f"Diseases with Test ROC-AUC > 0.65: {len(results_df[results_df['Test_ROC_AUC'] > 0.65])}")
    print(f"Diseases meeting criteria: {len(good_models)}")
    print(f"\nMean CV ROC-AUC: {results_df['CV_ROC_AUC_Mean'].mean():.4f} ± {results_df['CV_ROC_AUC_Mean'].std():.4f}")
    print(f"Mean Test ROC-AUC: {results_df['Test_ROC_AUC'].mean():.4f} ± {results_df['Test_ROC_AUC'].std():.4f}")
    print(f"Mean Overfitting Gap: {results_df['Overfit_Gap'].mean():.4f} ± {results_df['Overfit_Gap'].std():.4f}")
    
    if len(good_models) > 0:
        print("\n" + "="*80)
        print("TOP 10 BEST MODELS (by Nested CV Performance)")
        print("="*80)
        display_cols = ['Disease', 'CV_ROC_AUC_Mean', 'CV_ROC_AUC_Std', 
                       'Test_ROC_AUC', 'Test_F1', 'Overfit_Gap', 'N_Features_Selected']
        print(good_models[display_cols].head(10).to_string(index=False))
        
        # Save good models
        good_models.to_csv('good_classification_models_nested_cv.csv', index=False)
        logger.info("\n✓ Saved: good_classification_models_nested_cv.csv")
        
        # ====================================================================
        # Visualization 1: Nested CV performance comparison
        # ====================================================================
        top_n = min(20, len(good_models))
        top_models = good_models.head(top_n)
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Nested CV ROC-AUC with error bars
        y_pos = np.arange(len(top_models))
        axes[0, 0].barh(y_pos, top_models['CV_ROC_AUC_Mean'], 
                       xerr=top_models['CV_ROC_AUC_Std'],
                       color='steelblue', alpha=0.8, capsize=3)
        axes[0, 0].set_yticks(y_pos)
        axes[0, 0].set_yticklabels(top_models['Disease'], fontsize=9)
        axes[0, 0].set_xlabel('Nested CV ROC-AUC (mean ± std)', fontweight='bold')
        axes[0, 0].set_title('Nested Cross-Validation Performance', fontweight='bold')
        axes[0, 0].grid(axis='x', alpha=0.3)
        axes[0, 0].axvline(0.65, color='red', linestyle='--', alpha=0.5, label='0.65 threshold')
        axes[0, 0].invert_yaxis()
        axes[0, 0].legend()
        
        # Test vs CV comparison
        x_pos = np.arange(len(top_models))
        width = 0.35
        axes[0, 1].bar(x_pos - width/2, top_models['CV_ROC_AUC_Mean'], width, 
                      label='Nested CV', color='steelblue', alpha=0.8)
        axes[0, 1].bar(x_pos + width/2, top_models['Test_ROC_AUC'], width,
                      label='Test Set', color='coral', alpha=0.8)
        axes[0, 1].set_xticks(x_pos)
        axes[0, 1].set_xticklabels(top_models['Disease'], rotation=45, ha='right', fontsize=8)
        axes[0, 1].set_ylabel('ROC-AUC', fontweight='bold')
        axes[0, 1].set_title('CV vs Test Performance Comparison', fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(axis='y', alpha=0.3)
        
        # Overfitting gap
        axes[1, 0].barh(y_pos, top_models['Overfit_Gap'], 
                       color=['green' if x < 0.1 else 'orange' if x < 0.15 else 'red' 
                              for x in top_models['Overfit_Gap']], alpha=0.8)
        axes[1, 0].set_yticks(y_pos)
        axes[1, 0].set_yticklabels(top_models['Disease'], fontsize=9)
        axes[1, 0].set_xlabel('Overfitting Gap (Train AUC - CV AUC)', fontweight='bold')
        axes[1, 0].set_title('Overfitting Assessment (from Nested CV)', fontweight='bold')
        axes[1, 0].grid(axis='x', alpha=0.3)
        axes[1, 0].axvline(0.1, color='green', linestyle='--', alpha=0.5, label='Good (<0.1)')
        axes[1, 0].axvline(0.15, color='orange', linestyle='--', alpha=0.5, label='Acceptable (<0.15)')
        axes[1, 0].invert_yaxis()
        axes[1, 0].legend()
        
        # Selected features vs performance
        scatter = axes[1, 1].scatter(top_models['N_Features_Selected'], 
                                    top_models['CV_ROC_AUC_Mean'],
                                    c=top_models['Overfit_Gap'], 
                                    cmap='RdYlGn_r', s=100, alpha=0.8)
        
        # Add disease labels
        for i, row in top_models.iterrows():
            axes[1, 1].annotate(row['Disease'][:15], 
                               (row['N_Features_Selected'], row['CV_ROC_AUC_Mean']),
                               fontsize=7, alpha=0.7)
        
        axes[1, 1].set_xlabel('Number of Selected Features', fontweight='bold')
        axes[1, 1].set_ylabel('CV ROC-AUC', fontweight='bold')
        axes[1, 1].set_title('Features vs Performance', fontweight='bold')
        axes[1, 1].grid(alpha=0.3)
        
        # Add colorbar for overfitting gap
        cbar = plt.colorbar(scatter, ax=axes[1, 1])
        cbar.set_label('Overfitting Gap', fontweight='bold')
        
        plt.suptitle('XGBoost Classification with Nested CV and Improved Feature Selection', 
                    fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.savefig('classification_performance_nested_cv.png', dpi=300, bbox_inches='tight')
        logger.info("✓ Saved: classification_performance_nested_cv.png")
        plt.close()
        
        # ====================================================================
        # NEW: SHAP Analysis Summary for Best Models
        # ====================================================================
        print("\n[9/9] Creating SHAP analysis summary for top models...")
        
        # Create summary of SHAP findings
        shap_summary = []
        for _, row in good_models.head(5).iterrows():
            disease = row['Disease']
            if disease in shap_explanations:
                shap_data = shap_explanations[disease]
                shap_vals = shap_data['shap_values']
                
                # Calculate mean absolute SHAP values
                mean_abs_shap = np.abs(shap_vals).mean(0)
                top_features_idx = np.argsort(mean_abs_shap)[-5:][::-1]
                top_features = [shap_data['feature_names'][i] for i in top_features_idx]
                top_shap_values = mean_abs_shap[top_features_idx]
                
                shap_summary.append({
                    'Disease': disease,
                    'CV_AUC': row['CV_ROC_AUC_Mean'],
                    'Top_Feature_1': top_features[0],
                    'Top_SHAP_1': top_shap_values[0],
                    'Top_Feature_2': top_features[1],
                    'Top_SHAP_2': top_shap_values[1],
                    'Top_Feature_3': top_features[2],
                    'Top_SHAP_3': top_shap_values[2]
                })
        
        if shap_summary:
            shap_df = pd.DataFrame(shap_summary)
            shap_df.to_csv('shap_top_features_summary.csv', index=False)
            logger.info("✓ Saved: shap_top_features_summary.csv")
            
            print("\n" + "="*80)
            print("TOP BIOMARKERS IDENTIFIED BY SHAP")
            print("="*80)
            print(shap_df[['Disease', 'CV_AUC', 'Top_Feature_1', 'Top_Feature_2', 'Top_Feature_3']].to_string(index=False))
        
    else:
        print("\n⚠ WARNING: No diseases met the criteria (CV AUC > 0.65)")
        print("This is realistic for microbiome classification tasks.")
        print("\nSummary of all models (sorted by CV AUC):")
        print("="*80)
        display_cols = ['Disease', 'CV_ROC_AUC_Mean', 'CV_ROC_AUC_Std', 
                       'Test_ROC_AUC', 'Overfit_Gap', 'N_Disease', 'N_Features_Selected']
        print(results_df[display_cols].head(20).to_string(index=False))

2026-02-05 16:34:49,446 - INFO - ✓ Saved: xgboost_classification_results_nested_cv.csv
2026-02-05 16:34:49,447 - INFO - 
✓ Found 31 diseases with good nested CV performance
2026-02-05 16:34:49,447 - INFO -   (CV AUC > 0.65, Overfitting gap < 0.15)
2026-02-05 16:34:49,451 - INFO - 
✓ Saved: good_classification_models_nested_cv.csv



[8/9] Analyzing results...

PERFORMANCE SUMMARY (NESTED CV)
Total diseases analyzed: 35
Diseases with CV ROC-AUC > 0.65: 35
Diseases with Test ROC-AUC > 0.65: 34
Diseases meeting criteria: 31

Mean CV ROC-AUC: 0.8949 ± 0.0747
Mean Test ROC-AUC: 0.8932 ± 0.0878
Mean Overfitting Gap: 0.0853 ± 0.0535

TOP 10 BEST MODELS (by Nested CV Performance)
                  Disease  CV_ROC_AUC_Mean  CV_ROC_AUC_Std  Test_ROC_AUC  Test_F1  Overfit_Gap  N_Features_Selected
  Kidney Failure, Chronic         0.992658        0.003258      0.992208 0.909091     0.007342                   45
                   Asthma         0.989291        0.007036      0.954221 0.846154     0.010709                  145
            Healthy Aging         0.980112        0.011975      0.983437 0.925000     0.019888                  179
         Anorexia Nervosa         0.974488        0.023143      0.933107 0.800000     0.025512                  123
     Diabetic Retinopathy         0.972695        0.016655      0.968832 

2026-02-05 16:34:50,984 - INFO - ✓ Saved: classification_performance_nested_cv.png
2026-02-05 16:34:50,987 - INFO - ✓ Saved: shap_top_features_summary.csv



[9/9] Creating SHAP analysis summary for top models...

TOP BIOMARKERS IDENTIFIED BY SHAP
                Disease   CV_AUC      Top_Feature_1     Top_Feature_2 Top_Feature_3
Kidney Failure, Chronic 0.992658      Streptococcus   Parabacteroides     Dialister
                 Asthma 0.989291 Mediterraneibacter Lachnoclostridium     Roseburia
          Healthy Aging 0.980112       Anaerostipes      unclassified   Bacteroides
       Anorexia Nervosa 0.974488   Colidextribacter       Bacteroides  Anaerostipes
   Diabetic Retinopathy 0.972695      Brevundimonas       Pseudomonas  Anaerostipes


In [52]:
# ============================================================================
# 9. FINAL SUMMARY
# ============================================================================
print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nKey Files Generated:")
print("  1. xgboost_classification_results_nested_cv.csv - Complete results")
print("  2. good_classification_models_nested_cv.csv - Models passing quality criteria")
print("  3. healthy_samples_umap.png - Healthy cohort visualization")
print("  4. combined_healthy_disease_umap.png - All samples visualization")
print("  5. classification_performance_nested_cv.png - Performance overview")
print("  6. shap_summary_*.png - SHAP summary plots for good models")
print("  7. shap_bar_*.png - SHAP bar plots for feature importance")
print("  8. shap_dependence_*.png - SHAP dependence plots")
print("  9. shap_top_features_summary.csv - Summary of top biomarkers")
print(" 10. *_umap_3D.html - Interactive 3D visualizations")

print("\n" + "="*80)
print("IMPORTANT IMPROVEMENTS IMPLEMENTED:")
print("="*80)
print("1. NESTED CROSS-VALIDATION: Eliminates data leakage completely")
print("   - Inner CV: Feature selection + hyperparameter tuning")
print("   - Outer CV: Unbiased performance estimation")
print("2. IMPROVED FEATURE SELECTION:")
print("   - Mutual information for compositional data")
print("   - Correlation filtering to remove redundancy")
print("   - Integrated into pipeline for no-leakage")
print("3. MODEL INTERPRETABILITY:")
print("   - SHAP values for feature importance")
print("   - Summary plots, bar plots, dependence plots")
print("   - Identifies top biomarkers for each disease")
print("4. REALISTIC PERFORMANCE METRICS:")
print("   - Nested CV provides unbiased estimates")
print("   - Lower but more trustworthy AUC values")
print("="*80)

logger.info("\nScript completed successfully with nested CV and SHAP interpretability!")

2026-02-05 16:34:50,994 - INFO - 
Script completed successfully with nested CV and SHAP interpretability!



ANALYSIS COMPLETE!

Key Files Generated:
  1. xgboost_classification_results_nested_cv.csv - Complete results
  2. good_classification_models_nested_cv.csv - Models passing quality criteria
  3. healthy_samples_umap.png - Healthy cohort visualization
  4. combined_healthy_disease_umap.png - All samples visualization
  5. classification_performance_nested_cv.png - Performance overview
  6. shap_summary_*.png - SHAP summary plots for good models
  7. shap_bar_*.png - SHAP bar plots for feature importance
  8. shap_dependence_*.png - SHAP dependence plots
  9. shap_top_features_summary.csv - Summary of top biomarkers
 10. *_umap_3D.html - Interactive 3D visualizations

IMPORTANT IMPROVEMENTS IMPLEMENTED:
1. NESTED CROSS-VALIDATION: Eliminates data leakage completely
   - Inner CV: Feature selection + hyperparameter tuning
   - Outer CV: Unbiased performance estimation
2. IMPROVED FEATURE SELECTION:
   - Mutual information for compositional data
   - Correlation filtering to remove redund